# CoverCraft - Cover Letter Automation Script

## Setup
### Import Required Libraries

In [11]:
import os
import json

In [12]:
# api_key = os.getenv("OPENAI_API_KEY")

### Define Utility Functions

In [2]:
def read_file(file_path):
    with open(file_path) as f:
        return f.readlines()

# Read job_info.txt,profile_info.txt,letter_raw.txt files
# job_info = read_file("jobs/example.txt")
# profile_info = read_file("profile_info.txt")
# letter_raw = read_file("letter_raw.txt")

## OpenAI API Integration
### Function to create API Completion

In [3]:
def create_openai_completion(model, messages):
    """Calls the OpenAI API to generate text completions."""
    from openai import OpenAI
    client = OpenAI()
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature= 0.15,
            response_format={ "type": "json_object" }
        )
        return completion
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Prepare Prompt for API

In [4]:
def prepare_messages(profile_info, job_info, letter_format):
    return [
        {"role": "system", "content": "Je suis un assistant qui permet de remplir les lettres de motivation pour vos candidatures. Je fais le lien entre vos expériences, vos compétences et l'offre d'emploi. Le schema JSON doit contenir : Entreprise, Poste, Objet, Contenu"},
        {"role": "user", "content": f"Voici mes compétences et expériences : {profile_info}"},
        {"role": "system", "content": "Donnez moi l'offre d'emploi."},
        {"role": "user", "content": f"Voici le contenu de l'offre d'emploi : {job_info}"},
        {"role": "system", "content": "Donne moi le format type de lettre de motivation que je dois suivre."},
        {"role": "user", "content": f"Voici le format type de contenu que j'aimerais que tu suives : {letter_format}"},
    ]

## Application Processing

### Process a Single Job Application

In [6]:
def process_job_application(profile_info, letter_raw, job_info_filename):
    """Processes a single job application."""
    job_info = read_file(job_info_filename)
    job_name = os.path.splitext(os.path.basename(job_info_filename))[0]
    print(f"Processing: {job_name}")
    
    messages = prepare_messages(profile_info, job_info, letter_raw)
    # completion = create_openai_completion("gpt-4-0125-preview", messages)
    completion = create_openai_completion("gpt-3.5-turbo-0125", messages)
    

    if completion:
        result_raw = json.loads(completion.choices[0].message.content)
        # print(result_raw) # DEBUG

        # Prepare export content
        link_to_the_job = job_info[0]
        export_content = f"{link_to_the_job}\n\n{result_raw['Objet']}\n{result_raw['Contenu']}"
        
        # Define export path
        file_path = f"CL/{result_raw['Entreprise']} - {result_raw['Poste'].replace('/', '-')}.txt"

        
        # Write the content to file
        with open(file_path, "w", encoding='utf-8') as file:
            file.write(export_content)
        print(f"Exported to {file_path}")
    else:
        print("Failed to get a response from the OpenAI API.")
    print("============================================\n")

### Process All Job Applications

In [9]:
def process_all_jobs(jobs_directory, profile_info, letter_raw):
    """Processes all job applications in a directory."""
    for filename in os.listdir(jobs_directory):
        if filename.startswith("job_info_") and filename.endswith(".txt"):
            job_info_filename = os.path.join(jobs_directory, filename)
            # print(job_info_filename) # DEBUG
            
            process_job_application(profile_info, letter_raw, job_info_filename)

## Main Execution
### Execute Application Processing

In [10]:
data_directory = "data" 
profile_info_path = os.path.join(data_directory, "profile_info.txt")
letter_raw_path = os.path.join(data_directory, "letter_raw.txt")
jobs_directory = os.path.join(data_directory, "jobs")

# Read the common files
profile_info = read_file(profile_info_path)
letter_raw = read_file(letter_raw_path)

# Process all job applications
process_all_jobs(jobs_directory, profile_info, letter_raw)

Processing: job_info_1
Exported to CL/Bouygues Telecom - Développeur IA-Test.txt

Processing: job_info_2
Exported to CL/Bouygues Telecom - Alternant(e) - Analyste Développeur IA & Data F-H.txt

Processing: job_info_3
Exported to CL/Thales - Développement logiciel - Data Science - Construction et déploiement d'indicateurs.txt

